# TCN time series forecasting

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from lightning.pytorch import seed_everything, Trainer

from ts_utils import SineCosineDataModule, TCN

In [ ]:
_ = seed_everything(42)

## Generate data

In [ ]:
datamodule = SineCosineDataModule(
    num_steps=2000,
    max_length=100.,
    noise_level=0.02,
    random_seed=42,
    train_size=0.6,
    val_size=0.2,
    test_size=0.2,
    window_size=24,
    mode='next',
    next_steps=1,
    time_last=True,
    batch_size=32,
    num_workers=0
)

datamodule.prepare_data()
datamodule.setup(stage='fit')
datamodule.setup(stage='test')

print(f'Train data shape: {datamodule.train_data.shape}')
print(f'Val. data shape: {datamodule.val_data.shape}')
print(f'Test data shape: {datamodule.test_data.shape}')

print(f'\nNum. train batches: {len(datamodule.train_dataloader())}')
print(f'Num. val. batches: {len(datamodule.val_dataloader())}')
print(f'Num. test batches: {len(datamodule.test_dataloader())}')

In [ ]:
x_batch, y_batch = next(iter(datamodule.train_dataloader()))

print(f'Input shape: {x_batch.shape}')
print(f'Target shape: {y_batch.shape}')

In [ ]:
data = datamodule.data.numpy()

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(6, 4))
for idx, ax in enumerate(axes.ravel()):
    ax.plot(np.arange(len(data)) + 1, data[:,idx], color=plt.cm.Dark2(idx), alpha=0.7)
    ax.set(xlabel='t', ylabel=f'$y_{idx + 1}$')
    ax.set_xlim((0, len(data)))
    ax.grid(color='lightgray', linestyle='-')
    ax.set_axisbelow(True)
fig.tight_layout()

## Construct model

In [ ]:
model = TCN(
    num_channels=[2, 4, 4, 2],
    kernel_size=6,
    bias=True,
    weight_norm=False,
    activation='relu',
    activate_last=False,
    dropout=None,
    loss='mse',
    lr=1e-04
)

print(f'Num. weights: {model.num_weights()}')
print(model)

In [ ]:
x_batch, y_batch = next(iter(datamodule.train_dataloader()))

y_pred = model.forecast(x_batch)

print(f'Input shape: {x_batch.shape}')
print(f'Target shape: {y_batch.shape}')
print(f'Predict shape: {y_pred.shape}')

## Load model


In [ ]:
ckpt_path = '../run/tcn/version_0/checkpoints/best.ckpt'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = TCN.load_from_checkpoint(ckpt_path, map_location=device)
model = model.eval()

In [ ]:
trainer = Trainer(
    logger=False,
    accelerator='auto',
    enable_checkpointing=False
)

metrics = trainer.test(
    model,
    datamodule=datamodule,
    verbose=True
)

## Test predictions

In [ ]:
test_data = datamodule.test_data
window_size = datamodule.window_size

seq = torch.as_tensor(test_data[:window_size])  # (steps, features)
seq = seq.T.unsqueeze(0)  # (batch=1, features, steps)

preds = model.forecast_iteratively(
    seq.to(model.device), steps=len(test_data) - window_size
).cpu()  # (batch=1, features, steps)

seq = seq[0].numpy()
preds = preds[0].numpy()

print(f'Input shape: {seq.shape}')
print(f'Predict shape: {preds.shape}')

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(6, 4))
for idx, ax in enumerate(axes.ravel()):
    ax.plot(
        np.arange(window_size) + 1, seq[idx],
        color=plt.cm.Dark2(idx), alpha=0.7, label='test init'
    )
    ax.plot(
        np.arange(window_size, len(test_data)) + 1, preds[idx],
        color=plt.cm.Dark2(idx), alpha=0.7, linestyle='--', label='forecast'
    )
    ax.set(xlabel='t', ylabel=f'$y_{idx + 1}$')
    ax.set_xlim((0, len(test_data)))
    ax.legend(loc='upper right')
    ax.grid(color='lightgray', linestyle='-')
    ax.set_axisbelow(True)
fig.tight_layout()